<a href="https://colab.research.google.com/github/Mohammedmutar/OCTs/blob/main/Efficient_Vision_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=e8530acdb3f99cc67193ba0bfbf75971e7b1be0f984b27ca28c90062f2afb5ad
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.core.display import Image
import tensorflow as tf
from tensorflow.keras import backend as K
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow import keras
from sklearn.manifold import TSNE
from tensorflow.keras.models import Model
import seaborn as sns
import lime
import lime.lime_image
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import cv2
import random

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data_dir = '/content/drive/MyDrive/OCTT/oct_train'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=15,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2 )

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=False,
    class_mode='categorical',
    subset='validation'
)

Found 1570 images belonging to 6 classes.
Found 389 images belonging to 6 classes.


In [4]:

callbacks = [
    ModelCheckpoint(
        filepath='best_model_auc.h5',
        monitor='val_acc',
        mode='max',
        save_best_only=True,
        verbose=1),
    EarlyStopping(
        monitor='val_acc',
        mode='max',
        patience=5,
        restore_best_weights=True,
        verbose=1),
    ReduceLROnPlateau(
        monitor='val_acc',
        mode='max',
        factor=0.5,
        patience=2,
        verbose=1,
        min_lr=1e-6)]

In [7]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
labels = train_generator.classes
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)
# compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)


class_weights_dict = dict(enumerate(class_weights))



In [8]:


def focal_loss(gamma=2.0, alpha=0.25):

    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        cross_entropy = -y_true * K.log(y_pred)

        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        return K.sum(loss, axis=1)

    return focal_loss_fixed


In [9]:
def PatchEmbedding(inputs, patch_size=4, embed_dim=96):
    x = layers.Conv2D(embed_dim, kernel_size=patch_size, strides=patch_size, padding="same")(inputs)
    x = layers.LayerNormalization()(x)
    return x

def MultiHeadSelfAttention(inputs, num_heads):
    x = layers.LayerNormalization()(inputs)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(x, x)
    return layers.Add()([inputs, x])

def TransformerBlock(inputs, num_heads, mlp_dim):
    x = MultiHeadSelfAttention(inputs, num_heads)
    y = layers.LayerNormalization()(x)
    y = layers.Dense(mlp_dim, activation="gelu")(y)
    y = layers.Dense(inputs.shape[-1])(y)
    return layers.Add()([x, y])

def build_vit_base(input_shape=(224, 224, 3), num_classes=6):
    inputs = keras.Input(shape=input_shape)
    x = PatchEmbedding(inputs, patch_size=16, embed_dim=768)

    x = layers.Reshape((-1, 768))(x)

    for _ in range(6):
        x = TransformerBlock(x, num_heads=12, mlp_dim=3072)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs, name="ViT-Base")

def build_deit_base(input_shape=(224, 224, 3), num_classes=6):
    inputs = keras.Input(shape=input_shape)
    x = PatchEmbedding(inputs, patch_size=16, embed_dim=768)

    x = layers.Reshape((-1, 768))(x)

    for _ in range(6):
        x = TransformerBlock(x, num_heads=8, mlp_dim=3072)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs, name="DeiT-Base")

def build_maxvit_l(input_shape=(224, 224, 3), num_classes=6):
    inputs = keras.Input(shape=input_shape)
    x = PatchEmbedding(inputs, patch_size=16, embed_dim=768)

    for _ in range(4):
        x = TransformerBlock(x, num_heads=16, mlp_dim=4096)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs, name="MaxViT-L")

models_dict = {
    "ViT-Base": build_vit_base(),
    "DeiT-Base": build_deit_base(),
    "MaxViT-L": build_maxvit_l()}
results = {}




In [ ]:
# Train Each Model
for model_name, model in models_dict.items():
    print(f"\n🔵 Training {model_name}...")

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  loss=focal_loss(),
                  metrics=["accuracy", 'AUC'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(train_generator,
        validation_data=val_generator,
        epochs=8,
        callbacks=[early_stopping])

    results[model_name] = history.history
    model.save(f"{model_name}_model.h5")

print("\n✅ Training complete!")


🔵 Training ViT-Base...
Epoch 1/8


In [7]:



true_labels = val_generator.classes
class_names = list(val_generator.class_indices.keys())

results = {}
idx = random.randint(0, len(val_generator.filenames) - 1)

for model_name, model in models_dict.items():
    print(f"\n🔵 Evaluating {model_name}...")
    model.compile (optimizer='adam', loss='categorical_crossentropy')

    predictions = model.predict(val_generator)
    predicted_classes = np.argmax(predictions, axis=1)

    accuracy = np.mean(predicted_classes == true_labels)
    print(f"✅ Test Accuracy for {model_name}: {accuracy:.4f}")

    results[model_name] = {
        "predictions": predicted_classes,
        "accuracy": accuracy,
        "true_labels": true_labels
    }

    cm = confusion_matrix(true_labels, predicted_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, cmap="Blues", xticklabels=class_names, yticklabels=class_names, fmt="d")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.show()

    # ✅ Classification Report
    print(f"📊 Classification Report for {model_name}:\n", classification_report(true_labels, predicted_classes, target_names=class_names))


    # ✅ Predicted vs True Values
    plt.figure(figsize=(8, 6))
    plt.scatter(true_labels, predicted_classes, alpha=0.6)
    plt.xlabel("True Labels")
    plt.ylabel("Predicted Labels")
    plt.title(f"Predicted vs True - {model_name}")
    plt.show()

    # ✅ LIME Visualization
    explainer = lime.lime_image.LimeImageExplainer()

    # random test image
    img_path = val_generator.filepaths[idx]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224)) / 255.0
    img = np.expand_dims(img, axis=0)

    # model prediction using LIME
    def model_predict(x):
        return model.predict(x)

    explanation = explainer.explain_instance(img[0].astype('double'), model_predict, top_labels=5, hide_color=0, num_samples=1000)

    # Show LIME result
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(img[0])
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(temp)
    plt.imshow(mask, alpha=0.5)
    plt.title("LIME Explanation")
    plt.axis("off")
    plt.show()

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def Transformer_GradCAM(model, img_array, attention_layer):
    # ✅ Find the correct layer
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(attention_layer).output, model.output]
    )

    with tf.GradientTape() as tape:
        attention_output, predictions = grad_model(img_array)
        class_idx = np.argmax(predictions[0])  # Get predicted class index
        loss = predictions[:, class_idx]  # Get loss for that class

    grads = tape.gradient(loss, attention_output)  # Compute gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))  # Pool gradients

    attention_output = attention_output.numpy()[0]  # Extract feature map
    heatmap = np.mean(attention_output * pooled_grads.numpy(), axis=-1)  # Compute attention map

    # Normalize heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap


# ✅ Plot Heatmap
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(img[0])
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(img[0])
plt.imshow(heatmap, cmap="jet", alpha=0.5)
plt.title("Transformer Attention Heatmap")
plt.axis("off")
plt.show()




🔵 Evaluating ViT-Base...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

In [24]:
image_directory = "/content/drive/MyDrive/OCTT/seg"

image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

df_unlabeled = pd.DataFrame({"filename": image_files})
df_unlabeled["label"] = None  # No labels yet

print(df_unlabeled.head())
print(f"Total images in DataFrame: {len(df_unlabeled)}")


                                            filename label
0  /content/drive/MyDrive/OCTT/seg/61906_3DHWide-...  None
1  /content/drive/MyDrive/OCTT/seg/{0C0A045A-16E8...  None
2  /content/drive/MyDrive/OCTT/seg/{084C4527-DEAA...  None
3  /content/drive/MyDrive/OCTT/seg/{01E6D9FE-BF65...  None
4  /content/drive/MyDrive/OCTT/seg/{06DD694D-C939...  None
Total images in DataFrame: 1739


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

unlabeled_generator = datagen.flow_from_dataframe(
    dataframe=df_unlabeled,
    x_col="filename",
    y_col=None,
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False)
print(f"✅ Generator is ready with {unlabeled_generator.samples} images!")


Found 1739 validated image filenames.
✅ Generator is ready with 1739 images!


In [ ]:


confidence_threshold = 0.99

output_dir = "/content/drive/MyDrive/OCTT/Pseudo_Labels"
os.makedirs(output_dir, exist_ok=True)

for model_name, model in models_dict.items():
    print(f"🔵 Predicting pseudo-labels using {model_name}...")

    # Generate predictions
    predictions = model.predict(unlabeled_generator, verbose=1)
    pseudo_labels = np.argmax(predictions, axis=1)

    # Apply Confidence Threshold
    confident_indices = np.max(predictions, axis=1) >= confidence_threshold
    selected_filenames = np.array(unlabeled_generator.filenames)[confident_indices]
    selected_labels = pseudo_labels[confident_indices]

    # Create DataFrame for Pseudo-Labeled Data
    df_pseudo = pd.DataFrame({"filename": selected_filenames, "label": selected_labels})

    save_path = os.path.join(output_dir, f"{model_name}_pseudo_labels.csv")
    df_pseudo.to_csv(save_path, index=False)

    print(f"✅ Saved {len(df_pseudo)} high-confidence pseudo-labeled images for {model_name} at {save_path}")


🔵 Predicting pseudo-labels using ViT-Base...
587/587 ━━━━━━━━━━━━━━━━━━━━ 337s 575ms/step
✅ Saved 5293 high-confidence pseudo-labeled images for ViT-Base at /content/drive/MyDrive/OCTT/Pseudo_Labels/ViT-Base_pseudo_labels.csv
🔵 Predicting pseudo-labels using DeiT-Base...
587/587 ━━━━━━━━━━━━━━━━━━━━ 336s 572ms/step
✅ Saved 9957 high-confidence pseudo-labeled images for DeiT-Base at /content/drive/MyDrive/OCTT/Pseudo_Labels/DeiT-Base_pseudo_labels.csv
🔵 Predicting pseudo-labels using MaxViT-L...
587/587 ━━━━━━━━━━━━━━━━━━━━ 333s 568ms/step
✅ Saved 8621 high-confidence pseudo-labeled images for MaxViT-L at /content/drive/MyDrive/OCTT/Pseudo_Labels/MaxViT-L_pseudo_labels.csv


In [12]:

labeled_data_directory = "/content/drive/MyDrive/OCTT/oct_train"  # Change to actual path

# Get class names (subfolder names) and sort them for consistent labeling
class_names = sorted(os.listdir(labeled_data_directory))  # Ensures correct order
class_mapping = {class_name: idx for idx, class_name in enumerate(class_names)}  # Assigns 0-5 labels


labeled_data = []


for class_name, label in class_mapping.items():
    class_folder = os.path.join(labeled_data_directory, class_name)


    for filename in os.listdir(class_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(class_folder, filename)
            labeled_data.append((file_path, label))


df_labeled = pd.DataFrame(labeled_data, columns=["filename", "label"])

print(f"✅ Found {len(df_labeled)} labeled images across {len(class_names)} classes (0-5)")
print("Class Mapping:", class_mapping)  # Show folder names and their assigned label
print(df_labeled.head())  # Display sample data


✅ Found 1959 labeled images across 6 classes (0-5)
Class Mapping: {'Anterior Segment oct': 0, 'FAF': 1, 'Fundus camera': 2, 'Macula oct': 3, 'OCT_angio': 4, 'ONH oct': 5}
                                            filename  label
0  /content/drive/MyDrive/OCTT/oct_train/Anterior...      0
1  /content/drive/MyDrive/OCTT/oct_train/Anterior...      0
2  /content/drive/MyDrive/OCTT/oct_train/Anterior...      0
3  /content/drive/MyDrive/OCTT/oct_train/Anterior...      0
4  /content/drive/MyDrive/OCTT/oct_train/Anterior...      0


In [13]:
import pandas as pd
import os

pseudo_labels_dir = "/content/drive/MyDrive/OCTT/Pseudo_Labels"
labeled_data_path = "/content/drive/MyDrive/OCTT/Labeled_Dataset.csv"

transformer_models = ["ViT-Base", "DeiT-Base", "MaxViT-L"]

# ✅ Loop through each transformer model and create a separate merged dataset
for model_name in transformer_models:
    pseudo_file = os.path.join(pseudo_labels_dir, f"{model_name}_pseudo_labels.csv")

    if os.path.exists(pseudo_file):
        df_pseudo = pd.read_csv(pseudo_file)
        df_pseudo["source_model"] = model_name  # Track which model generated the pseudo-labels

        # ✅ Merge labeled and pseudo-labeled data
        semi_supervised_data = pd.concat([df_labeled, df_pseudo], ignore_index=True)

        # ✅ Shuffle the dataset
        semi_supervised_data = semi_supervised_data.sample(frac=1).reset_index(drop=True)

        # ✅ Save the merged dataset
        merged_dataset_path = f"/content/drive/MyDrive/OCTT/SemiSupervised_{model_name}.csv"
        semi_supervised_data.to_csv(merged_dataset_path, index=False)

        print(f"✅ Created semi-supervised dataset with {len(semi_supervised_data)} images for {model_name}")
        print(f"✅ Saved at {merged_dataset_path}\n")


✅ Created semi-supervised dataset with 7252 images for ViT-Base
✅ Saved at /content/drive/MyDrive/OCTT/SemiSupervised_ViT-Base.csv

✅ Created semi-supervised dataset with 11916 images for DeiT-Base
✅ Saved at /content/drive/MyDrive/OCTT/SemiSupervised_DeiT-Base.csv

✅ Created semi-supervised dataset with 10580 images for MaxViT-L
✅ Saved at /content/drive/MyDrive/OCTT/SemiSupervised_MaxViT-L.csv



In [14]:


def train_model_on_pseudo_labels(model_name, base_model):
    """
    Train a given model on its pseudo-labeled dataset.
    """
    print(f"\n🚀 Training {model_name} on Semi-Supervised Data...")

    semi_supervised_data = pd.read_csv(pseudo_labeled_paths[model_name])

    semi_supervised_data["label"] = semi_supervised_data["label"].astype(str)

    train_df = semi_supervised_data.sample(frac=0.8, random_state=42)
    val_df = semi_supervised_data.drop(train_df.index)

    print(f"✅ Training set: {len(train_df)} images, Validation set: {len(val_df)} images")

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="filename",
        y_col="label",
        target_size=(224, 224),
        batch_size=8,
        class_mode="categorical",
        shuffle=True
    )

    val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col="filename",
        y_col="label",
        target_size=(224, 224),
        batch_size=8,
        class_mode="categorical"
    )

    print(f"✅ Data Generators Ready for {model_name}!")


    model = base_model

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    print(f"✅ {model_name} Model Compiled!")

    history = model.fit(
        train_generator,
        epochs=3,
                validation_data=val_generator,

        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
            tf.keras.callbacks.ModelCheckpoint(f"{model_name}_semi_supervised.h5", save_best_only=True, monitor="val_loss")
        ]
    )

    print(f"✅ {model_name} Training Complete!")

    return model, history, val_generator


In [18]:
pseudo_labeled_paths = {
    "ViT-Base": "/content/drive/MyDrive/OCTT/SemiSupervised_ViT-Base.csv",
    "DeiT-Base": "/content/drive/MyDrive/OCTT/SemiSupervised_DeiT-Base.csv",
    "MaxViT-L": "/content/drive/MyDrive/OCTT/SemiSupervised_MaxViT-L.csv",}
vit_model, vit_history, vit_val_gen = train_model_on_pseudo_labels("ViT-Base", build_vit_base())
deit_model, deit_history, deit_val_gen = train_model_on_pseudo_labels("DeiT-Base", build_deit_base())
maxvit_model, maxvit_history, maxvit_val_gen = train_model_on_pseudo_labels("MaxViT-L", build_maxvit_l())


🚀 Training ViT-Base on Semi-Supervised Data...
✅ Training set: 5802 images, Validation set: 1450 images
Found 1575 validated image filenames belonging to 6 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 4227 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 384 validated image filenames belonging to 6 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1066 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


✅ Data Generators Ready for ViT-Base!
✅ ViT-Base Model Compiled!
Epoch 1/3
 14/197 ━━━━━━━━━━━━━━━━━━━━ 3:29:47 69s/step - accuracy: 0.2108 - loss: 7.8825

KeyboardInterrupt: 

Features PCA


In [16]:
def load_transformer_for_feature_extraction(model_name):

    if model_name == "DeiT-Base":
        model = build_deit_base()
        model = keras.Model(inputs=model.input, outputs=model.layers[-4].output)

    elif model_name == "ViT-Base":
        model = build_vit_base()
        model = keras.Model(inputs=model.input, outputs=model.layers[-4].output)

    elif model_name == "MaxViT-L":
        model = build_maxvit_l()
        model = keras.Model(inputs=model.input, outputs=model.layers[-4].output)

    else:
        raise ValueError("Invalid model name")

    return model

# ✅ Load Transformer Models
models_dict = {
    "ViT-Base": load_transformer_for_feature_extraction("ViT-Base"),
    "DeiT-Base": load_transformer_for_feature_extraction("DeiT-Base"),
    "MaxViT-L": load_transformer_for_feature_extraction("MaxViT-L"),

}

# ✅ Extract Features
for model_name, model in models_dict.items():
    print(f"🔵 Extracting features using {model_name} from layer: {model.layers[-1].name}...")


🔵 Extracting features using ViT-Base from layer: global_average_pooling1d_3...
🔵 Extracting features using DeiT-Base from layer: global_average_pooling1d_4...
🔵 Extracting features using MaxViT-L from layer: global_average_pooling2d_1...


In [19]:

image_directory = "/content/drive/MyDrive/OCTT/All_OCTs"

def preprocess_image(image_path):
    """ Load and preprocess a single image for Transformer models. """
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image) / 255.0  # Normalize
    return image

def extract_features_from_unlabeled(model, image_paths):
    """Extract feature embeddings from a given model."""
    features = []

    for img_path in image_paths:
        img = preprocess_image(img_path)
        print (img_path)
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        # Extract features from the model
        feature_vector = model.predict(img, verbose=0)
        features.append(feature_vector.flatten())  # Flatten feature map

    return np.array(features)

image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

features_dict = {}

#  Extract features for each model
for model_name, model in models_dict.items():
    print(f"\n🔵 Extracting features using {model_name}...")
    features_dict[model_name] = extract_features_from_unlabeled(model, image_files)

#  PCA and KMeans Clustering
pca_kmeans_results = {}

for model_name, features in features_dict.items():
    print(f"\n🔵 Applying PCA & KMeans for {model_name}...")

    pca = PCA(n_components=50)
    features_pca = pca.fit_transform(features)

    kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(features_pca)

    pca_kmeans_results[model_name] = {
        "features_pca": features_pca,
        "clusters": cluster_labels
    }

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

model_names = [ "DeiT-Base", "ViT-Base", "MaxViT-L"]

for i, model_name in enumerate(model_names):
    features_pca = pca_kmeans_results[model_name]["features_pca"]
    clusters = pca_kmeans_results[model_name]["clusters"]

    scatter = axes[i].scatter(features_pca[:, 0], features_pca[:, 1], c=clusters, cmap="viridis", alpha=0.5)
    axes[i].set_title(f"PCA Clustering for {model_name}")
    axes[i].set_xlabel("PCA Component 1")
    axes[i].set_ylabel("PCA Component 2")

plt.tight_layout()
plt.show()




🔵 Extracting features using ViT-Base...

🔵 Extracting features using DeiT-Base...

🔵 Extracting features using MaxViT-L...

🔵 Applying PCA & KMeans for ViT-Base...


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

SimCLR


In [20]:
def build_clr_model(encoder, name="CLR_Model"):
    input_1 = keras.Input(shape=(224, 224, 3))
    input_2 = keras.Input(shape=(224, 224, 3))

    feat_1 = encoder(input_1)
    feat_2 = encoder(input_2)

    projection_head = keras.Sequential([
        layers.Dense(512, activation='relu'),
        layers.Dense(128)
    ])

    z_1 = projection_head(feat_1)
    z_2 = projection_head(feat_2)

    return keras.Model(inputs=[input_1, input_2], outputs=[z_1, z_2], name=name)
def load_transformer_encoder(path):
    model = keras.models.load_model(path)
    encoder = keras.Model(inputs=model.input, outputs=model.layers[-4].output)  # Assumes GlobalAvgPool is -4
    encoder.trainable = True
    return encoder

model_paths = {
    "ViT-Base": "/content/drive/MyDrive/OCTT/ViT-Base.h5",
    "DeiT-Base": "/content/drive/MyDrive/OCTT/DeiT-Base.h5",
    "MaxViT-L": "/content/drive/MyDrive/OCTT/MaxViT-L.h5"}

clr_models = {}
for name, path in model_paths.items():
    encoder = load_transformer_encoder(path)
    clr_models[name] = build_clr_model(encoder, name=f"{name}_CLR")

print(" CLR Models created for all Transformers.")


 CLR Models created for all Transformers.


In [21]:
def nt_xent_loss(z_i, z_j, temperature=0.5):
    z_i = tf.math.l2_normalize(z_i, axis=1)
    z_j = tf.math.l2_normalize(z_j, axis=1)
    similarities = tf.matmul(z_i, z_j, transpose_b=True) / temperature
    labels = tf.range(tf.shape(similarities)[0])
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels, similarities))


def train_clr_model(model, generator, epochs=1, temperature=0.5):
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)

    for epoch in range(epochs):
        print(f"\n🚀 Epoch {epoch+1}/{epochs}")
        for step in range(len(generator)):
            batch = next(iter(generator))
            batch_aug1 = batch.copy()
            batch_aug2 = batch.copy()

            with tf.GradientTape() as tape:
                proj_1 = model(batch_aug1, training=True)
                proj_2 = model(batch_aug2, training=True)
                loss = nt_xent_loss(proj_1, proj_2, temperature)

            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            if step % 10 == 1:
                print(f"🔄 Step {step+1}/{len(generator)} - Loss: {loss.numpy():.4f}")
    print("✅ CLR training completed.")
    return model  # ✅ Return the model!


In [ ]:
for idx, (model_name, model) in enumerate(models_dict.items(), start=1):
    print(f"\n🔧 Training Model {idx}: {model_name}")

    # Train model using your function
    trained_model = train_clr_model(model, unlabeled_generator)

    # Save the model
    save_path = f'/content/model_{idx}_{model_name}.h5'
    trained_model.save(save_path)
    print(f"✅ Saved as {save_path}")



🔧 Training Model 1: ViT-Base

🚀 Epoch 1/1


In [ ]:


image_directory = "/content/drive/MyDrive/OCTT/All_OCTs"
image_paths = [os.path.join(image_directory, fname) for fname in os.listdir(image_directory)
               if fname.lower().endswith(('.png', '.jpg', '.jpeg'))]  # Limit for t-SNE

# Extract from the 4th last layer (usually GAP before dense layers)
gap_layer_output = model12.layers[-4].output

# Build the feature extractor model
feature_extractor = Model(inputs=model12.input, outputs=gap_layer_output)

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    return np.expand_dims(img, axis=0)

def extract_features(model, image_paths):
    features = []
    for path in image_paths:
        img = preprocess_image(path)
        feature_vector = feature_extractor.predict(img, verbose=0)
        features.append(feature_vector.flatten())
    return np.array(features)


features = extract_features(feature_extractor, image_paths)

# ✅ PCA
pca = PCA(n_components=40)
pca_features = pca.fit_transform(features)

# ✅ t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
tsne_results = tsne.fit_transform(pca_features)

# ✅ Plotting
plt.figure(figsize=(10, 6))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], s=10, alpha=0.7, cmap='viridis')
plt.title("t-SNE Visualization of CLR Learned Embeddings (DeiT-Base)", fontsize=14)
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.grid(True)
plt.show()


In [ ]:
# Train Each Model
for idx, (model_name, model) in enumerate(models_dict.items(), start=1):
    print(f"\n🔵 Training {model_name}...")

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  loss=focal_loss(),
                  metrics=["accuracy", 'AUC'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    history = model.fit(train_generator,
        validation_data=val_generator,
        epochs=8,
        callbacks=[early_stopping])

    results[model_name] = history.history
    model.save(f"{model_name}_model.h5")

print("\n✅ Training complete!")

Full Train


In [ ]:


confidence_threshold = 0.99

output_dir = "/content/drive/MyDrive/OCTT/Final_Pseudo_Labels"
os.makedirs(output_dir, exist_ok=True)

for model_name, model in models_dict.items():
    print(f"🔵 Predicting pseudo-labels using {model_name}...")

    # Generate predictions
    predictions = model.predict(unlabeled_generator, verbose=1)
    pseudo_labels = np.argmax(predictions, axis=1)

    # Apply Confidence Threshold
    confident_indices = np.max(predictions, axis=1) >= confidence_threshold
    selected_filenames = np.array(unlabeled_generator.filenames)[confident_indices]
    selected_labels = pseudo_labels[confident_indices]

    # Create DataFrame for Pseudo-Labeled Data
    df_pseudo = pd.DataFrame({"filename": selected_filenames, "label": selected_labels})

    save_path = os.path.join(output_dir, f"{model_name}_pseudo_labels.csv")
    df_pseudo.to_csv(save_path, index=False)

    print(f"✅ Saved {len(df_pseudo)} high-confidence pseudo-labeled images for {model_name} at {save_path}")


🔵 Predicting pseudo-labels using ViT-Base...
587/587 ━━━━━━━━━━━━━━━━━━━━ 337s 575ms/step
✅ Saved 5293 high-confidence pseudo-labeled images for ViT-Base at /content/drive/MyDrive/OCTT/Pseudo_Labels/ViT-Base_pseudo_labels.csv
🔵 Predicting pseudo-labels using DeiT-Base...
587/587 ━━━━━━━━━━━━━━━━━━━━ 336s 572ms/step
✅ Saved 9957 high-confidence pseudo-labeled images for DeiT-Base at /content/drive/MyDrive/OCTT/Pseudo_Labels/DeiT-Base_pseudo_labels.csv
🔵 Predicting pseudo-labels using MaxViT-L...
587/587 ━━━━━━━━━━━━━━━━━━━━ 333s 568ms/step
✅ Saved 8621 high-confidence pseudo-labeled images for MaxViT-L at /content/drive/MyDrive/OCTT/Pseudo_Labels/MaxViT-L_pseudo_labels.csv


In [ ]:
import pandas as pd
import os

pseudo_labels_dir = "/content/drive/MyDrive/OCTT/Pseudo_Labels"
labeled_data_path = "/content/drive/MyDrive/OCTT/Labeled_Dataset.csv"

transformer_models = ["ViT-Base", "DeiT-Base", "MaxViT-L"]

# ✅ Loop through each transformer model and create a separate merged dataset
for model_name in transformer_models:
    pseudo_file = os.path.join(pseudo_labels_dir, f"{model_name}_pseudo_labels.csv")

    if os.path.exists(pseudo_file):
        df_pseudo = pd.read_csv(pseudo_file)
        df_pseudo["source_model"] = model_name  # Track which model generated the pseudo-labels

        # ✅ Merge labeled and pseudo-labeled data
        semi_supervised_data = pd.concat([df_labeled, df_pseudo], ignore_index=True)

        # ✅ Shuffle the dataset
        semi_supervised_data = semi_supervised_data.sample(frac=1).reset_index(drop=True)

        # ✅ Save the merged dataset
        merged_dataset_path = f"/content/drive/MyDrive/OCTT/SemiSupervisedCLR_{model_name}.csv"
        semi_supervised_data.to_csv(merged_dataset_path, index=False)

        print(f"✅ Created semi-supervised dataset with {len(semi_supervised_data)} images for {model_name}")
        print(f"✅ Saved at {merged_dataset_path}\n")


✅ Created semi-supervised dataset with 7252 images for ViT-Base
✅ Saved at /content/drive/MyDrive/OCTT/SemiSupervised_ViT-Base.csv

✅ Created semi-supervised dataset with 11916 images for DeiT-Base
✅ Saved at /content/drive/MyDrive/OCTT/SemiSupervised_DeiT-Base.csv

✅ Created semi-supervised dataset with 10580 images for MaxViT-L
✅ Saved at /content/drive/MyDrive/OCTT/SemiSupervised_MaxViT-L.csv



In [ ]:


def train_model_on_pseudo_labels(model_name, base_model):
    """
    Train a given model on its pseudo-labeled dataset.
    """
    print(f"\n🚀 Training {model_name} on Semi-Supervised Data...")

    semi_supervised_data = pd.read_csv(pseudo_labeled_paths[model_name])

    semi_supervised_data["label"] = semi_supervised_data["label"].astype(str)

    train_df = semi_supervised_data.sample(frac=0.8, random_state=42)
    val_df = semi_supervised_data.drop(train_df.index)

    print(f"✅ Training set: {len(train_df)} images, Validation set: {len(val_df)} images")

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="filename",
        y_col="label",
        target_size=(224, 224),
        batch_size=8,
        class_mode="categorical",
        shuffle=True
    )

    val_generator = val_datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col="filename",
        y_col="label",
        target_size=(224, 224),
        batch_size=8,
        class_mode="categorical"
    )

    print(f"✅ Data Generators Ready for {model_name}!")


    model = base_model

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    print(f"✅ {model_name} Model Compiled!")

    history = model.fit(
        train_generator,
        epochs=3,
                validation_data=val_generator,

        callbacks=[
            tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
            tf.keras.callbacks.ModelCheckpoint(f"{model_name}_semi_supervised.h5", save_best_only=True, monitor="val_loss")
        ]
    )

    print(f"✅ {model_name} Training Complete!")

    return model, history, val_generator


In [ ]:
pseudo_labeled_paths = {
    "ViT-Base": "/content/drive/MyDrive/OCTT/SemiSupervisedCLR_ViT-Base.csv",
    "DeiT-Base": "/content/drive/MyDrive/OCTT/SemiSupervisedCLR_DeiT-Base.csv",
    "MaxViT-L": "/content/drive/MyDrive/OCTT/SemiSupervisedCLR_MaxViT-L.csv",}
vit_model, vit_history, vit_val_gen = train_model_on_pseudo_labels("ViT-Base", build_vit_base())
deit_model, deit_history, deit_val_gen = train_model_on_pseudo_labels("DeiT-Base", build_deit_base())
maxvit_model, maxvit_history, maxvit_val_gen = train_model_on_pseudo_labels("MaxViT-L", build_maxvit_l())


🚀 Training ViT-Base on Semi-Supervised Data...
✅ Training set: 5802 images, Validation set: 1450 images
Found 1575 validated image filenames belonging to 6 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 4227 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 384 validated image filenames belonging to 6 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1066 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


✅ Data Generators Ready for ViT-Base!
✅ ViT-Base Model Compiled!
Epoch 1/3
 14/197 ━━━━━━━━━━━━━━━━━━━━ 3:29:47 69s/step - accuracy: 0.2108 - loss: 7.8825

KeyboardInterrupt: 

In [ ]:
def load_transformer_for_feature_extraction(model_name):

    if model_name == "DeiT-Base":
        model = build_deit_base()
        model = keras.Model(inputs=model.input, outputs=model.layers[-4].output)

    elif model_name == "ViT-Base":
        model = build_vit_base()
        model = keras.Model(inputs=model.input, outputs=model.layers[-4].output)

    elif model_name == "MaxViT-L":
        model = build_maxvit_l()
        model = keras.Model(inputs=model.input, outputs=model.layers[-4].output)

    else:
        raise ValueError("Invalid model name")

    return model

# ✅ Load Transformer Models
models_dict = {
    "ViT-Base": load_transformer_for_feature_extraction("ViT-Base"),
    "DeiT-Base": load_transformer_for_feature_extraction("DeiT-Base"),
    "MaxViT-L": load_transformer_for_feature_extraction("MaxViT-L"),

}

# ✅ Extract Features
for model_name, model in models_dict.items():
    print(f"🔵 Extracting features using {model_name} from layer: {model.layers[-1].name}...")


🔵 Extracting features using ViT-Base from layer: global_average_pooling1d_3...
🔵 Extracting features using DeiT-Base from layer: global_average_pooling1d_4...
🔵 Extracting features using MaxViT-L from layer: global_average_pooling2d_1...


In [ ]:

image_directory = "/content/drive/MyDrive/OCTT/All_OCTs"

def preprocess_image(image_path):
    """ Load and preprocess a single image for Transformer models. """
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image) / 255.0  # Normalize
    return image

def extract_features_from_unlabeled(model, image_paths):
    """Extract feature embeddings from a given model."""
    features = []

    for img_path in image_paths:
        img = preprocess_image(img_path)
        print (img_path)
        img = np.expand_dims(img, axis=0)

        feature_vector = model.predict(img, verbose=0)
        features.append(feature_vector.flatten())

    return np.array(features)

image_files = [os.path.join(image_directory, f) for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

features_dict = {}

for model_name, model in models_dict.items():
    print(f"\n🔵 Extracting features using {model_name}...")
    features_dict[model_name] = extract_features_from_unlabeled(model, image_files)

pca_kmeans_results = {}

for model_name, features in features_dict.items():
    print(f"\n🔵 Applying PCA & KMeans for {model_name}...")

    pca = PCA(n_components=50)
    features_pca = pca.fit_transform(features)

    kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(features_pca)

    pca_kmeans_results[model_name] = {
        "features_pca": features_pca,
        "clusters": cluster_labels
    }

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

model_names = [ "DeiT-Base", "ViT-Base", "MaxViT-L"]

for i, model_name in enumerate(model_names):
    features_pca = pca_kmeans_results[model_name]["features_pca"]
    clusters = pca_kmeans_results[model_name]["clusters"]

    scatter = axes[i].scatter(features_pca[:, 0], features_pca[:, 1], c=clusters, cmap="viridis", alpha=0.5)
    axes[i].set_title(f"PCA Clustering for {model_name}")
    axes[i].set_xlabel("PCA Component 1")
    axes[i].set_ylabel("PCA Component 2")

plt.tight_layout()
plt.show()




🔵 Extracting features using ViT-Base...

🔵 Extracting features using DeiT-Base...

🔵 Extracting features using MaxViT-L...

🔵 Applying PCA & KMeans for ViT-Base...


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.